<a href="https://colab.research.google.com/github/Roudranil/competitions/blob/main/dphi/datasprint%2324/Data_Sprint_24_hyper_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [36]:
X = pd.read_csv('https://raw.githubusercontent.com/Roudranil/competitions/main/dphi/datasprint%2324/X.csv', index_col=0)
y = np.ravel(pd.read_csv('https://raw.githubusercontent.com/Roudranil/competitions/main/dphi/datasprint%2324/y.csv', index_col=0))
test = pd.read_csv('https://raw.githubusercontent.com/Roudranil/competitions/main/dphi/datasprint%2324/test.csv', index_col=0)
submission = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/Tinder_Millennial_Match/sample_submission.csv')

In [21]:
s1 = pd.read_csv('/content/submission2.csv', names=['xgb'])
s2 = pd.read_csv('/content/submission8.csv', names=['rf1'])
s3 = pd.read_csv('/content/submission4.csv', names=['rf2'])
s = pd.concat([s1, s2, s3], axis=1)
s = s.drop(index=0)
s = s.astype(int)

In [22]:
s['diff_atleast1'] = np.abs(s['xgb'] - s['rf1'] + s['xgb'] - s['rf2'])

In [ ]:
s[s['diff_atleast1'] > 0]

# model
---

In [ ]:
!pip install optuna

In [54]:
!pip uninstall lightgbm

Uninstalling lightgbm-2.2.3:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/lightgbm-2.2.3.dist-info/*
    /usr/local/lib/python3.6/dist-packages/lightgbm/*
Proceed (y/n)? y
  Successfully uninstalled lightgbm-2.2.3


In [55]:
!pip install lightgbm --install-option=--gpu

/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
     |████████████████████████████████| 593kB 16.0MB/s 
Skipping wheel build for lightgbm, due to binaries being disabled for it.
    Running setup.py install for lightgbm ... done


In [4]:

from xgboost import XGBClassifier
import optuna
from optuna import Trial, visualization

In [5]:
from sklearn.model_selection import KFold, cross_val_score

In [56]:
from lightgbm import LGBMClassifier

In [6]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [51]:
def Objective(trial):
    params = {
        "n_estimators": trial.suggest_categorical("n_estimators", [90,100,125,150,175,200,225]),
        "eta": trial.suggest_loguniform("eta",1e-2,0.1),
        "subsample": trial.suggest_discrete_uniform("subsample", 0.7,1,0.05),
        "min_child_weight": trial.suggest_int("min_child_weight",1,12),
        "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree", 0,1,0.05),
        "max_depth": trial.suggest_categorical("max_depth",[3,4,5,7,9,11]),
        "base_score": trial.suggest_loguniform("base_score", 0.3, 0.5),
        "random_state": 2021
    }
    
    model = XGBClassifier(**params, tree_method='gpu_hist')
    
    
    scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
    
    return sum(scores)/len(scores)

In [52]:
study = optuna.create_study(direction="maximize", study_name='Xgboost optimization')
study.optimize(Objective, n_trials=40)

[I 2021-02-13 18:07:03,671] A new study created in memory with name: Xgboost optimization
[I 2021-02-13 18:07:07,859] Trial 0 finished with value: 0.9176306347810306 and parameters: {'n_estimators': 200, 'eta': 0.03670757204128871, 'subsample': 0.75, 'min_child_weight': 8, 'colsample_bytree': 0.9, 'max_depth': 3, 'base_score': 0.30243330082933007}. Best is trial 0 with value: 0.9176306347810306.
[I 2021-02-13 18:07:09,907] Trial 1 finished with value: 0.9186874397956192 and parameters: {'n_estimators': 90, 'eta': 0.09677112480819501, 'subsample': 1.0, 'min_child_weight': 6, 'colsample_bytree': 0.45, 'max_depth': 4, 'base_score': 0.4169622917155342}. Best is trial 1 with value: 0.9186874397956192.
[I 2021-02-13 18:07:12,998] Trial 2 finished with value: 0.9113009730423979 and parameters: {'n_estimators': 150, 'eta': 0.02744609900946724, 'subsample': 1.0, 'min_child_weight': 4, 'colsample_bytree': 0.1, 'max_depth': 3, 'base_score': 0.3379990971768708}. Best is trial 1 with value: 0.91868

In [53]:
study.best_params

{'base_score': 0.31458509669541074,
 'colsample_bytree': 0.6000000000000001,
 'eta': 0.018439538618899406,
 'max_depth': 3,
 'min_child_weight': 2,
 'n_estimators': 90,
 'subsample': 0.7}

In [60]:
XGB_params = study.best_params

In [61]:
def Objective(trial):
    params = {
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
    
    model = LGBMClassifier(**params, device_type='GPU')
    scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')
    
    return sum(scores)/len(scores)

In [62]:
study = optuna.create_study(direction="maximize", study_name='LGBM optimization')
study.optimize(Objective, n_trials=40)

[I 2021-02-13 18:21:47,079] A new study created in memory with name: LGBM optimization


[LightGBM] [Warning] feature_fraction is set=0.5847569979917171, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5847569979917171
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] lambda_l2 is set=0.0003007187689105893, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0003007187689105893
[LightGBM] [Warning] lambda_l1 is set=0.00010256085623167177, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00010256085623167177
[LightGBM] [Warning] bagging_fraction is set=0.5268641914914194, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5268641914914194
[LightGBM] [Warning] feature_fraction is set=0.5847569979917171, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5847569979917171
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] lambda_l2 is set=0.00030071

[I 2021-02-13 18:21:48,999] Trial 0 finished with value: 0.9186916279264565 and parameters: {'lambda_l1': 0.00010256085623167177, 'lambda_l2': 0.0003007187689105893, 'num_leaves': 161, 'feature_fraction': 0.5847569979917171, 'bagging_fraction': 0.5268641914914194, 'bagging_freq': 2, 'min_child_samples': 61}. Best is trial 0 with value: 0.9186916279264565.


[LightGBM] [Warning] feature_fraction is set=0.6402782005291808, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6402782005291808
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=0.00317809931278659, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.00317809931278659
[LightGBM] [Warning] lambda_l1 is set=1.5733706297029726, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.5733706297029726
[LightGBM] [Warning] bagging_fraction is set=0.7825654082552838, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7825654082552838
[LightGBM] [Warning] feature_fraction is set=0.6402782005291808, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6402782005291808
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=0.00317809931278659, r

[I 2021-02-13 18:21:50,412] Trial 1 finished with value: 0.9171029302955425 and parameters: {'lambda_l1': 1.5733706297029726, 'lambda_l2': 0.00317809931278659, 'num_leaves': 71, 'feature_fraction': 0.6402782005291808, 'bagging_fraction': 0.7825654082552838, 'bagging_freq': 5, 'min_child_samples': 87}. Best is trial 0 with value: 0.9186916279264565.


...

[I 2021-02-13 18:22:46,362] Trial 37 finished with value: 0.9123577780569864 and parameters: {'lambda_l1': 0.06673638222412126, 'lambda_l2': 0.004633429754983468, 'num_leaves': 52, 'feature_fraction': 0.8324466243685166, 'bagging_fraction': 0.9561067079909004, 'bagging_freq': 7, 'min_child_samples': 91}. Best is trial 35 with value: 0.9239714648685624.


[LightGBM] [Warning] feature_fraction is set=0.9656771868660532, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9656771868660532
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] lambda_l2 is set=0.0005786755863040118, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0005786755863040118
[LightGBM] [Warning] lambda_l1 is set=1.4611720054421837, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4611720054421837
[LightGBM] [Warning] bagging_fraction is set=0.8865407475018441, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8865407475018441
[LightGBM] [Warning] feature_fraction is set=0.9656771868660532, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9656771868660532
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] lambda_l2 is set=0.0005786755863040

[I 2021-02-13 18:22:47,794] Trial 38 finished with value: 0.9181639234409683 and parameters: {'lambda_l1': 1.4611720054421837, 'lambda_l2': 0.0005786755863040118, 'num_leaves': 23, 'feature_fraction': 0.9656771868660532, 'bagging_fraction': 0.8865407475018441, 'bagging_freq': 7, 'min_child_samples': 80}. Best is trial 35 with value: 0.9239714648685624.


[LightGBM] [Warning] feature_fraction is set=0.8143651742329092, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8143651742329092
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] lambda_l2 is set=0.0001135402576339061, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0001135402576339061
[LightGBM] [Warning] lambda_l1 is set=9.610572880172, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.610572880172
[LightGBM] [Warning] bagging_fraction is set=0.957805811751374, subsample=1.0 will be ignored. Current value: bagging_fraction=0.957805811751374
[LightGBM] [Warning] feature_fraction is set=0.8143651742329092, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8143651742329092
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] lambda_l2 is set=0.0001135402576339061, reg_l

[I 2021-02-13 18:22:49,051] Trial 39 finished with value: 0.9244991693540505 and parameters: {'lambda_l1': 9.610572880172, 'lambda_l2': 0.0001135402576339061, 'num_leaves': 62, 'feature_fraction': 0.8143651742329092, 'bagging_fraction': 0.957805811751374, 'bagging_freq': 6, 'min_child_samples': 85}. Best is trial 39 with value: 0.9244991693540505.


In [63]:
lgb_params = study.best_params
study.best_params

{'bagging_fraction': 0.957805811751374,
 'bagging_freq': 6,
 'feature_fraction': 0.8143651742329092,
 'lambda_l1': 9.610572880172,
 'lambda_l2': 0.0001135402576339061,
 'min_child_samples': 85,
 'num_leaves': 62}

In [43]:
xgb = XGBClassifier(tree_method='gpu_hist')
scores = cross_val_score(xgb, X, y, cv=kf, scoring='accuracy')
scores.mean()

In [58]:
lgb = LGBMClassifier(device_type='gpu')
scores = cross_val_score(lgb, X, y, cv=kf, scoring='accuracy')
scores.mean()

0.9033812176292386

In [59]:
lgb.fit(X, y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               device_type='gpu', importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

# submission
---

In [27]:
submission['prediction'] = s1.drop(index=0)
submission.iloc[132, 0] = 1
submission.iloc[190, 0] = 1
submission.iloc[444, 0] = 0
submission.iloc[478, 0] = 1
submission.iloc[609, 0] = 1

In [46]:
submission['prediction'] = xgb.predict(test)
submission.iloc[444, 0] = 0
# submission.to_csv('submission11.csv', index=False)

In [47]:
submission['prediction'].mean()

0.310126582278481